In [1]:
import os
import pandas as pd

In [2]:
def read_imp_val_csv(file):
    """Read impossible.csv like file
    :param: filename
    :type: string
    :return: dictionary associating param and set of impossible values
    :rtype: dict
    """
    df = pd.read_csv(file)
    df_dict = df.to_dict()
    res = dict()
    for k, v in df_dict['option'].items():
        imp = map(lambda x: x.replace("'", ''), 
                  df_dict['impossible_value'][k][1:-1].split()) 
        res[v] = set(imp)
    return res

# Build the dict of impossible values
impossible_values = read_imp_val_csv('examples/impossible.csv')

In [ ]:
SPE_CONFIG='../linux-4.14.152/spe.config'
CONFIG_FILE='../linux-4.14.152/.config'
CONFIG_DIR='../linux-4.14.152'

test_dict = {'OMAP_SSI': {'m', 'y'},
 'FB_MB862XX_LIME': {'y'},
 'MTD_PXA2XX': {'m', 'y'}}

def check(n):
    """
    Forcing some impossible values then generate a .config
    Check n times for each options"""
    for k, v in impossible_values.items():
        print('-', k)
        for o in v:
            print('\t*', o)
            for i in range(n):
                cmd_ret = []
                line = 'CONFIG_{}={}'.format(k, o)
                with open(SPE_CONFIG, 'w') as stream:
                    stream.write(line + '\n')
                os.system("KCONFIG_ALLCONFIG=spe.config make -C {} randconfig"
                          .format(CONFIG_DIR))
                ret = os.popen("grep '{}' {}".format(line, CONFIG_FILE)).read()
                if(ret):
                    cmd_ret.append(ret)
            if cmd_ret:
                for r in cmd_ret:
                    print('\t\t', r)
            else:
                print('\t\t[OK]')

In [ ]:
check(4)